In [1]:
import numpy as np
import pandas as pd

In [2]:
gpt3 = pd.read_csv("/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv")
gpt4 = pd.read_csv("/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv")
palm = pd.read_csv("/kaggle/input/llm-generated-essay-using-palm-from-google-gen-ai/LLM_generated_essay_PaLM.csv")
persuade = pd.read_csv("/kaggle/input/persuade2-0/persuade_2.0_human_scores_demo_id_github.csv")
daigtv3 = pd.read_csv("/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_02.csv")
hello = pd.read_csv("/kaggle/input/hello-claude-1000-essays-from-anthropic/persuade15_claude_instant1.csv")

In [3]:
daigtv3

,text,label,prompt_name,source,RDizzl3_seven,model
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,human
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,human
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,human
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,human
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,human
...,...,...,...,...,...,...
65262,"Dear Senator,\n\nI am writing to you regarding...",1,Does the electoral college work?,text-davinci-003,True,davinci
65263,I remember the day distinctively. I was sittin...,1,Does the electoral college work?,text-davinci-003,True,davinci
65264,"Dear Senator, \n\nI am writing this letter to ...",1,Does the electoral college work?,text-davinci-003,True,davinci
65265,"Dear Senator,\n\nI am writing to urge you to e...",1,Does the electoral college work?,text-davinci-003,True,davinci


In [4]:
# 'Does the electoral college work?' sütunundan 712 adet ve 'Car-free cities' sütunundan 672 adet satırı seç
selected_rows_electoral = persuade[persuade['prompt_name'] == 'Does the electoral college work?'].head(712)
selected_rows_car_free = persuade[persuade['prompt_name'] == 'Car-free cities'].head(672)

# Yeni değişkenlere atayın
electoral = selected_rows_electoral.copy()
carfree = selected_rows_car_free.copy()

In [5]:
daigtv3 = daigtv3.drop("prompt_name", axis=1)
daigtv3 = daigtv3.drop("source", axis=1)
daigtv3 = daigtv3.drop("RDizzl3_seven", axis=1)
daigtv3 = daigtv3.drop("model", axis=1)

In [6]:
columns_to_drop = ['essay_id_comp', 'holistic_essay_score', 'word_count', 'prompt_name', 'task', 'assignment', 'source_text', 'gender', 'grade_level', 'ell_status', 'race_ethnicity', 'economically_disadvantaged', 'student_disability_status']

carfree = carfree.drop(columns_to_drop, axis=1)
electoral = electoral.drop(columns_to_drop, axis=1)

In [7]:
carfree['generated'] = 0
electoral['generated'] = 0

In [8]:
carfree

,full_text,generated
1168,Cars have been around for awhile and they have...,0
1169,Have you ever thought what it would be like no...,0
1170,What you are about to read is going to give yo...,0
1171,cars have many flaws nd and in this day and ag...,0
1172,There are many advantages of limiting car usag...,0
...,...,...
1835,Cars use a lot of gas and make the world pollu...,0
1836,With the advancement in technology in our soci...,0
1837,I think that not having cars is a good thing f...,0
1838,People use them to commute around various loca...,0


In [9]:
carfree.rename(columns={'full_text': 'text'}, inplace=True)
electoral.rename(columns={'full_text': 'text'}, inplace=True)
daigtv3.rename(columns={'label': 'generated'}, inplace=True)

In [10]:
electoral

,text,generated
18585,"Hey state senator, I'm writing this letter bec...",0
18586,The electoral college process consist of the s...,0
18587,I say we keep the Electoral College. For one w...,0
18588,So a letter to my state senator into keeping t...,0
18589,Many people are in belief of evaporating the i...,0
...,...,...
19292,"Dear, senator the Electorial College should be...",0
19293,"Dear Senator,\n\nMy name is PROPER_NAME from L...",0
19294,"Dear, State Senator\n\nHi, State Senator my na...",0
19295,The Electoral College is a system where electe...,0


In [11]:
sub = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv")
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")

In [12]:
# Koşulları sağlayan satırları filtreleyip kaldıralım
train_essays = train_essays[(train_essays['id'] != '82131f68') & (train_essays['id'] != '86fe4f18')]

In [13]:
baskanlik_secimi = train_essays[train_essays['prompt_id'] == 1]
baskanlik_secimi

,id,prompt_id,text,generated
5,00da8c32,1,The electrol college system is an unfair syste...,0
6,011dc2bc,1,"Dear state senator, It is the utmost respect t...",0
8,01c6e176,1,"""It's official: The electoral college is unfai...",0
9,0202ddf9,1,The Electoral College has been kept for centur...,0
10,020a5d6d,1,"Dear senator, Retain the Electoral College. Th...",0
...,...,...,...,...
1360,fc66f374,1,The Electoral College was originally establish...,0
1361,fcb87d59,1,"Dear senator, I think that the presidential el...",0
1364,fcd93e2d,1,The electoral college is a group of electors t...,0
1366,fcfe84cb,1,An electoral College compromises between elect...,0


In [14]:
cars = train_essays[train_essays['prompt_id'] == 0]
cars

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1372,fe5b0a93,0,studies have shown that passenger cars are res...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [15]:
columns_to_drop = ['id',"prompt_id"]

palm = palm.drop(columns_to_drop, axis=1)
gpt4 = gpt4.drop(columns_to_drop, axis=1)
gpt3 = gpt3.drop(columns_to_drop, axis=1)
cars = cars.drop(columns_to_drop, axis=1)
baskanlik_secimi = baskanlik_secimi.drop(columns_to_drop, axis=1)

In [16]:
cars = cars.iloc[:350, :]
baskanlik_secimi = baskanlik_secimi.iloc[:350, :]

In [17]:
cars

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
660,Fellow citizens should know about the advantag...,0
661,Have you ever realized how driving everywhere ...,0
662,A day without driving your car? who can't do t...,0
663,"In America, cars are considered an important n...",0


In [18]:
tekbir = pd.concat([baskanlik_secimi, cars, gpt3, gpt4, palm, electoral, carfree, daigtv3])

In [19]:
tekbir

,text,generated
5,The electrol college system is an unfair syste...,0.0
6,"Dear state senator, It is the utmost respect t...",0.0
8,"""It's official: The electoral college is unfai...",0.0
9,The Electoral College has been kept for centur...,0.0
10,"Dear senator, Retain the Electoral College. Th...",0.0
...,...,...
65262,"Dear Senator,\n\nI am writing to you regarding...",1.0
65263,I remember the day distinctively. I was sittin...,1.0
65264,"Dear Senator, \n\nI am writing this letter to ...",1.0
65265,"Dear Senator,\n\nI am writing to urge you to e...",1.0


In [20]:
tekbir.generated.value_counts()

generated
1.0    39981
0.0    29454
Name: count, dtype: int64

In [21]:
print(tekbir.isnull().sum())

text         0
generated    0
dtype: int64


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    tekbir.text,
    tekbir.generated,
    test_size=0.20,
    random_state=2024,
    stratify=tekbir.generated
)

In [23]:
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)

Shape of X_train (55548,)
Shape of X_test (13887,)


In [24]:
y_train.value_counts()

generated
1.0    31985
0.0    23563
Name: count, dtype: int64

In [25]:
y_test.value_counts()

generated
1.0    7996
0.0    5891
Name: count, dtype: int64

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

# TfidfVectorizer kullanarak metinleri vektörize etme (eğitim seti üzerinde fit işlemi)
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

# RandomForestClassifier kullanarak model oluşturma
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_vectorized, y_train)

# Test seti üzerinde tahmin yapma
X_test_vectorized = vectorizer.transform(X_test)
y_pred_test = rf_classifier.predict(X_test_vectorized)

# Sınıflandırma raporunu yazdırma
print(classification_report(y_test, y_pred_test))


              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      5891
         1.0       0.99      0.97      0.98      7996

    accuracy                           0.98     13887
   macro avg       0.98      0.98      0.98     13887
weighted avg       0.98      0.98      0.98     13887



In [27]:
X_test[:5]

11693    I would argue yes this could be a valueable. T...
51230     Facial Action Coding System (FACS) is a power...
32844    [Your Grade]\n[Your School]\n[Date]\n\nDear Pr...
59949    Driverless Cars are Coming, an intriguing arti...
32834    Sure, I'd be happy to write an essay as an 8th...
Name: text, dtype: object

In [28]:
y_test[:5]

11693    0.0
51230    1.0
32844    1.0
59949    1.0
32834    1.0
Name: generated, dtype: float64

In [29]:
y_pred_test[:5]

array([0., 1., 1., 1., 1.])

In [30]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Sınıflandırma raporunu yazdırma
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred_test))

# Diğer performans metriklerini yazdırma
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, rf_classifier.predict_proba(X_test_vectorized)[:, 1])

print("\nDiğer Performans Metrikleri:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC-AUC Score:", roc_auc)

Sınıflandırma Raporu:
               precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      5891
         1.0       0.99      0.97      0.98      7996

    accuracy                           0.98     13887
   macro avg       0.98      0.98      0.98     13887
weighted avg       0.98      0.98      0.98     13887


Diğer Performans Metrikleri:
Accuracy: 0.9785410815870959
Precision: 0.9914453524004085
Recall: 0.9711105552776388
F1 Score: 0.981172605509224
ROC-AUC Score: 0.9977926919664212


In [31]:
from sklearn.metrics import mean_squared_error, r2_score

# Test seti üzerinde tahmin yapma
X_test_vectorized = vectorizer.transform(X_test)
y_pred_test = rf_classifier.predict(X_test_vectorized)

# Mean Squared Error (MSE) ve R-squared (R2) skorlarını hesaplama
mse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

# Sonuçları yazdırma
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 0.021458918412904156
R-squared (R2): 0.9121457265723338


In [32]:
# Şimdi aynı özellik matriksini ve modeli test setine uygulayalım
zumba = test_essays['text']
zumba_vectorized = vectorizer.fit_transform(zumba)

In [33]:
zumba_vectorized.toarray()

array([[0.72033345, 0.54783215, 0.42544054, 0.        , 0.        ],
       [0.        , 0.61980538, 0.48133417, 0.61980538, 0.        ],
       [0.        , 0.        , 0.42544054, 0.54783215, 0.72033345]])

In [34]:
test_essays

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [35]:
tekbir

,text,generated
5,The electrol college system is an unfair syste...,0.0
6,"Dear state senator, It is the utmost respect t...",0.0
8,"""It's official: The electoral college is unfai...",0.0
9,The Electoral College has been kept for centur...,0.0
10,"Dear senator, Retain the Electoral College. Th...",0.0
...,...,...
65262,"Dear Senator,\n\nI am writing to you regarding...",1.0
65263,I remember the day distinctively. I was sittin...,1.0
65264,"Dear Senator, \n\nI am writing this letter to ...",1.0
65265,"Dear Senator,\n\nI am writing to urge you to e...",1.0


In [36]:
# TfidfVectorizer'ı eğitim verisi üzerinde uygula
vectorizer.fit(X_train)

# Test verisini vektörleştir
zumba_vectorized = vectorizer.transform(zumba)

# Tahmin yap
pred = rf_classifier.predict_proba(zumba_vectorized)[:, 1]

In [37]:
pred

array([1., 1., 1.])

In [38]:
# test_result = pred[0]
test_result = pred[:]
test_result

array([1., 1., 1.])

In [39]:
final_submission = pd.DataFrame(test_essays['id'])
final_submission['generated'] = test_result
final_submission

,id,generated
0,0000aaaa,1.0
1,1111bbbb,1.0
2,2222cccc,1.0


In [40]:
final_submission.to_csv('submission.csv', index=False)